# Preparing the dataset

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

ds = load_dataset("dmitva/human_ai_generated_text")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model_training_dataset.csv:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [ ]:
human_records = []
ai_records = []

for index, record in enumerate(ds['train']):

  if index > 100:
    break

  hmn_record = {
      "text": record['human_text'],
      "label": 0
  }

  ai_record = {
      "text": record['ai_text'],
      "label": 1
  }

  human_records.append(hmn_record)
  ai_records.append(ai_record)

In [ ]:
import pandas as pd

In [ ]:
len(ai_records)

101

In [ ]:
df = pd.DataFrame(human_records+ai_records)

In [ ]:
df.to_csv('human_ai_text.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import torch
import joblib
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Training and fitting the Model

In [ ]:
# df = pd.read_csv("human.csv")

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

count = 0

# Function to extract BERT embeddings
def get_bert_embeddings(text):
  global count
  count += 1
  tokens = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
  print(count)
  with torch.no_grad():
      outputs = bert_model(**tokens)
  return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Prepare dataset
X = df["text"].astype(str)
y = df["label"]

# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Generate BERT embeddings
X_train_bert = np.array([get_bert_embeddings(text) for text in X_train])
X_test_bert = np.array([get_bert_embeddings(text) for text in X_test])

logreg_bert = LogisticRegression()
logreg_bert.fit(X_train_bert, y_train)

# Evaluate Model
y_pred_bert = logreg_bert.predict(X_test_bert)
print("BERT-based Model Performance:")
print(classification_report(y_test, y_pred_bert))

# Save the trained model
joblib.dump(logreg_bert, "bert_ai_text_detector.pkl")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
BERT-based Model Performance:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98        22
           1       0.95      1.00      0.97        19

    accuracy                           0.98        41
   macro avg       0.97      0.98      0.98        4

# Testing the Model with example

In [ ]:
text = """For example, students need time to think whether or not if it's the right class for them. Every year in high school students are required to pick three choice to pick their class. Imagine getting a job that you're most interested and enjoys the most that later on life, you became super rich or making decent money but doesn't mind as long as you enjoy it and still continuing on. This way we can understand the situation and whether or not if the people and their families to decide if it's a good idea.

Students can gain little experience by taking the class every each year, this way students can learn more about the Jobs they want to be in. I also think this is very important. If you take swimming class every year, and gain a little bit of experience, you will be able to win the swimming contest. All this reason is to proof of why gaining experience is a good thing and help you understand the job you wanted to applied for.

Giving students the time to think on what jobs their most interested on is also very important for this prompt. Planning is really important for all of our lives. I really like this idea, it helps us understand what we're interested on and what to learn. Life can be hard if people don't enjoys their jobs, causing them to have depression. It's like, warming up for a swimming contest but, how can you win if you don't know how to swim? Swimming class exist to help people to not drown. It is called the Course Election, where students gets to pick three class that are interesting for them or forced to pick because it is a requirement. Many students have successfully acquired the jobs they intended to work because of how much work they put into to plan and learning the stuff they need to know on how to run the job. Because of that, planning has a big impact in our lives and helps us to decide on what jobs to do and to give us the time to think. For example, the military uses a plan and tactics in order for them to calculate and for the plan to be successful. That is why, students should select to visit and learn about the work that interests them because, it gives them more time to think, making plans for their future, and to gain experience through out the school years.

Students should be planning for their future and to decide which class to take in order for them to learn about the work that interests them. They exist because, it helps students to understand what its like for them to experience JROTC. For example, JROTC are similar to military but trained to be a marching men and to juggle with rifle (from what I know of). I think three years is enough for students to plan and to have a little experience for their future.

In conclusion, student are more successful if they are to be picked for themselves instead of the school choosing for them, because the more they like the class and the job they are going for, the harder they will try to succeed."""

In [ ]:
text = get_bert_embeddings(text)

205


In [ ]:
classes = ['human', 'ai']

In [ ]:
classes[logreg_bert.predict(text.reshape(1, -1))[0]]

'human'